In [81]:
# Importing libraries
import win32com.client as win32
import pandas as pd

In [82]:
# Loading email data from .csv to dataframe
df = pd.read_csv('Email_Details.csv')
display(df)

,Recipient(s),Subject(s),Body,Attachment Path(s),Attachment Status
0,test-recipient@gmail.com,Test-subject,"Hi,\n I hope all is well.\n\nI was just wri...",NaN,No
1,test-recipient-2@outlook.co.uk,Test-subject-2,"Hi,\n I hope all is well.\n\nI was just wri...",NaN,NaN
2,test-recipient-3@hotmail.com,Test-subject-3,"Hello,\n This is a test.\n\nThanks,\nX",NaN,NaN


In [83]:
# Seperating the email content
outlook = win32.Dispatch('outlook.application')

Recipients = df['Recipient(s)'].tolist()
Subjects = df['Subject(s)'].tolist()
Body = df['Body'].tolist()
Attachments = df['Attachment Path(s)'].tolist()  # Paths to attachments
Attach_status = df['Attachment Status'][0]

# Defining functions
def length_check(Recipients, Subjects, Body, Attachments, Attach_status):
    '''This function checks that the recipient, subject, body and attachment data are
    of compatible lengths. This means that either all have the same length (E.G 8 emails
    with each defined) or some are length 1, at which point they will be repeated for 
    every email later (E.G the same Body to many different recipients)'''
    
    # We first check whether attachments should be included
    if Attach_status=='Yes':
        lists = [Subjects, Body, Attachments]
    else:
        lists = [Subjects, Body]

    # We then check the lengths, and raise errors if necessary
    for l in lists:
        if len(l)==len(Recipients) or len(l)==1:
            pass
        elif len(l)>=len(Recipients):
            raise Exception("There is more email content than recipients!")
        else:
            raise Exception("Cannot associate the correct Subject, Body or Attachment content with each email")
        

def send_email(Recipients, Subjects, Body, Attachments, Attach_status):
    '''This function sends the emails, which involves creating each email item,
    checking the compatibility (as above), repeating data if necessary and then
    sending each email.'''
    
    # We check the lengths for each email input
    length_check(Recipients, Subjects, Body, Attachments, Attach_status)
    
    # Here we create and send each email
    for i in range(len(Recipients)):
        
        mail = outlook.CreateItem(0)
        
        # Filling each email with the correct data or repeating some content if only given once
        mail.To = Recipients[0] if len(Recipients) == 1 else Recipients[i]
        mail.Subject = Subjects[0] if len(Subjects) == 1 else Subjects[i]
        mail.Body = Body[0] if len(Body) == 1 else Body[i]    
        
        # Optionally adding attachments
        if Attach_status == 'Yes':
            mail.Attachments.Add(Attachments[0]) if len(Attachments) == 1 else Attachments[i]
            
        #mail.Display(True)
        mail.Send()

In [84]:
# Running the code
send_email(Recipients, Subjects, Body, Attachments, Attach_status)